In [ ]:
'''
Scrape county results for each presidential election from wiki and put into csv format
* Need to add 1988_Louisiana manually since it wasn't parsed correctly
* Alaska results are not displayed on wiki - will only use statewide results as training data for AK
'''

In [ ]:
## Set up the variables
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

years = [1976, 1980, 1984, 1988, 1992, 1996]
fips_file = '/Users/mattg/Personal Projects/2024_forecast/Data/fips_info.csv'

df = pd.read_csv(fips_file)
states = df['StateName'].unique()

# To replace spaces with underscores for links
states = [state.replace(' ', '_') for state in states]
states = [state.replace('District_of_Columbia', 'the_District_of_Columbia') for state in states]
states = [state.replace('Washington', 'Washington_(state)') for state in states]

In [ ]:
## OPTIONAL: Check that each url exists
for yr in years:  
    for state in states:    
        res_url = f'https://en.wikipedia.org/wiki/{yr}_United_States_presidential_election_in_{state}'
        response = requests.get(res_url)
        print(f"{yr}: {state}")
        # Check if the URL returns a 200 response code, indicating the page is accessible
        if response.status_code != 200:        
            print(f"{yr}: {state}")
            
print("Done!")

In [ ]:
## OPTIONAL: Displays which states and years have a county table
for yr in years:  
    for state in states:    
        res_url = f'https://en.wikipedia.org/wiki/{yr}_United_States_presidential_election_in_{state}'
        response = requests.get(res_url)
        
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'class': 'wikitable sortable'})
        
        if table:
            # Find the header row
            header_row = table.find('tr')
            if header_row:
                # Extract headers
                headers = [th.get_text(strip=True) for th in header_row.find_all('th')]
                # Check if any header contains the word "County"
                contains_county = any(("County" or "Parish") in header for header in headers)
                print(f"{yr}, {state.replace('_', ' ')}, {contains_county}")
            else:
                print(f"{yr}, {state.replace('_', ' ')}, NO COUNTY")
                print(res_url)
        else:
            print(f"{yr}, {state.replace('_', ' ')}, NO TABLE")
            print(res_url)

In [ ]:
## Places every table into a df, and each df into a dictionary
results = {}
for yr in years:
    for state in states:
        print(f"{yr}: {state}")
        res_url = f'https://en.wikipedia.org/wiki/{yr}_United_States_presidential_election_in_{state}'
        response = requests.get(res_url)
        
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', class_='wikitable sortable')
        
        if table:
            df = pd.read_html(str(table))[0]
            results[f"{yr}_{state}"] = df

del results['1988_Louisiana'] # Needs to be added manually


In [ ]:
## Flatten the dfs
for df in results.values():
    df.columns = ['_'.join(col) for col in df.columns]

## Filter out for only necessary columns
# DFL was added since the wiki page for Minnesota lists Dem candidates as DFL
pattern = re.compile('|'.join(['DFL', 'Democratic', 'Republican', 'Total']), flags=re.IGNORECASE)

# Keep the first column and filter the rest
for key, df in results.items():
    first_col = df.iloc[:, 0]
    filtered_df = df[df.columns[df.columns.str.contains(pattern, regex=True)]]
    cols = filtered_df.columns
    for col in cols:
        if '%' in col:
            filtered_df.drop(col, axis=1, inplace=True)
    results[key] = pd.concat([first_col, filtered_df], axis=1)
            
## Rename columns
for df in results.values():
    cols = df.columns
    df.rename(columns = {cols[0]: 'county', cols[3]:'total'}, inplace=True)
    if ('Democratic' or 'DFL') in cols[1]:
        df.rename(columns = {cols[1]: 'dem_votes', cols[2]: 'rep_votes'}, inplace=True)
    else:
        df.rename(columns = {cols[2]: 'dem_votes', cols[1]: 'rep_votes'}, inplace=True)
        
## LA needed to be added manually, bs4 won't scrape the info from wiki correctly
results['1988_Louisiana'] = pd.read_csv('/Users/mattg/Personal Projects/2024_forecast/Data/election_results/1988_Louisiana_pres_county.csv')


In [ ]:
## Example
results['1996_Alabama'].head(60)

In [ ]:
## Save this info as multiple csv
county_folder_path = '/Users/mattg/Personal Projects/2024_forecast/Data/election_results/county_results'

'''
for df in results.keys():
    results[df].to_csv(f'{county_folder_path}/{df}.csv', index=False)  
'''
